In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from sklearn.utils import resample
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import os

SOURCE_CSV_FILE = os.path.abspath("./datasets/dataset-for-bank-loan-prediction.csv")
NUMBER_CREDIT_PRO = "Number of Credit Problems"
# Đọc dữ liệu từ file CSV
data = pd.read_csv(SOURCE_CSV_FILE)

# Hiển thị những dòng đầu tiên của dữ liệu
print("Head of data:")
print(data.head())

# Kiểm tra kích thước dữ liệu
print(f"Number of rows: {data.shape[0]}")
print(f"Number of columns: {data.shape[1]}")

# Hiển thị thông tin về dữ liệu
print("Data Info:")
print(data.info())

# Hiển thị các thống kê mô tả
print("Data Description:")
print(data.describe(include='all'))

# Kiểm tra số lượng giá trị bị thiếu trong mỗi cột
missing_values = data.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Tính toán tỷ lệ giá trị bị thiếu
missing_percentage = (missing_values / data.shape[0]) * 100
print("Percentage of missing values per column:")
print(missing_percentage)

# Biểu đồ boxplot cho các cột số để phát hiện giá trị ngoại lai
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
# for col in numeric_columns:
#     plt.figure(figsize=(10, 6))
#     sns.boxplot(x=data[col])
#     plt.title(f'Boxplot for {col}')
#     plt.show()

# Tách các cột số và các cột không phải số
print("^^^^^^^^^^^^^^^^^^^ Tách các cột không phải số")
numeric_data = data[numeric_columns]
non_numeric_data = data.drop(columns=numeric_columns)

# print("^^^^^^^^^^^^^^^^^^^ Xử lý missing values")
# Xử lý missing values: Điền giá trị bằng phương pháp KNN imputation cho các cột số
# imputer = KNNImputer(n_neighbors=5)
# numeric_data_imputed = pd.DataFrame(imputer.fit_transform(numeric_data), columns=numeric_data.columns)
print("^^^^^^^^^^^^^^^^^^^ Sử dụng IterativeImputer")
# Sử dụng IterativeImputer
imputer = IterativeImputer(max_iter=10, random_state=0)
numeric_data_imputed = pd.DataFrame(imputer.fit_transform(numeric_data), columns=numeric_data.columns)

# Kết hợp lại dữ liệu sau khi điền các giá trị bị thiếu
print("^^^^^^^^^^^^^^^^^^^ Kết hợp lại dữ liệu sau khi điền các giá trị bị thiếu")
data_imputed = pd.concat([numeric_data_imputed, non_numeric_data.reset_index(drop=True)], axis=1)

print("^^^^^^^^^^^^^^^^^^^ Xử lý outliers: Winsorization")
# Xử lý outliers: Winsorization
for col in numeric_columns:
    data_imputed[col] = winsorize(data_imputed[col], limits=[0.05, 0.05])

print("^^^^^^^^^^^^^^^^^^^ Sửa giá trị không hợp lệ")
# Sửa giá trị không hợp lệ
# Number of Credit Problems chỉ nhận giá trị 0 và 1
data_imputed.loc[~data_imputed[NUMBER_CREDIT_PRO].isin([0, 1]), NUMBER_CREDIT_PRO] = np.nan
data_imputed[NUMBER_CREDIT_PRO].fillna(data_imputed[NUMBER_CREDIT_PRO].mode()[0], inplace=True)

print("^^^^^^^^^^^^^^^^^^^ Chuẩn hóa cột text về dạng chữ thường")
# Chuẩn hóa cột text về dạng chữ thường
text_columns = data_imputed.select_dtypes(include=['object']).columns
for col in text_columns:
    data_imputed[col] = data_imputed[col].str.lower()

# Tăng cường lớp NUMBER_CREDIT_PRO bằng kỹ thuật resampling
print("^^^^^^^^^^^^^^^^^^^ Tăng cường lớp NUMBER_CREDIT_PRO bằng kỹ thuật resampling")
if NUMBER_CREDIT_PRO in data_imputed.columns:
    churn_data = data_imputed[data_imputed[NUMBER_CREDIT_PRO] == 1]
    non_churn_data = data_imputed[data_imputed[NUMBER_CREDIT_PRO] == 0]
    churn_data_upsampled = resample(churn_data, replace=True, n_samples=len(non_churn_data), random_state=123)
    data_balanced = pd.concat([non_churn_data, churn_data_upsampled])
else:
    data_balanced = data_imputed

# Kiểm tra lại missing values
print("^^^^^^^^^^^^^^^^^^^ Missing values after imputation:")
print(data_balanced.isnull().sum())

# Kiểm tra lại outliers bằng boxplot
# for col in numeric_columns:
#     plt.figure(figsize=(10, 6))
#     sns.boxplot(x=data_balanced[col])
#     plt.title(f'Boxplot for {col} after Winsorization')
#     plt.show()

# Kiểm tra lại tính nhất quán và hợp lệ của dữ liệu
print("^^^^^^^^^^^^^^^^^^^ Unique values in NUMBER_CREDIT_PRO column after cleaning:")
print(data_balanced[NUMBER_CREDIT_PRO].unique())

# Lưu dữ liệu đã xử lý ra file CSV mới

CLEANTED_CSV_FILE = os.path.abspath("./datasets/cleaned/dataset-for-bank-loan-prediction_cleaned.csv")
data_balanced.to_csv(CLEANTED_CSV_FILE, index=False)
print("^^^^^^^^^^^^^^^^^^^ Cleaned data has been saved to 'cleaned/dataset-for-bank-loan-prediction_cleaned.csv'")

Head of data:
                                Loan ID                           Customer ID  \
0  14dd8831-6af5-400b-83ec-68e61888a048  981165ec-3274-42f5-a3b4-d104041a9ca9   
1  4771cc26-131a-45db-b5aa-537ea4ba5342  2de017a3-2e01-49cb-a581-08169e83be29   
2  4eed4e6a-aa2f-4c91-8651-ce984ee8fb26  5efb2b2b-bf11-4dfd-a572-3761a2694725   
3  77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a  e777faab-98ae-45af-9a86-7ce5b33b1011   
4  d4062e70-befa-4995-8643-a0de73938182  81536ad9-5ccf-4eb8-befb-47a4d608658e   

  Loan Status  Current Loan Amount        Term  Credit Score  Annual Income  \
0  Fully Paid             445412.0  Short Term         709.0      1167493.0   
1  Fully Paid             262328.0  Short Term           NaN            NaN   
2  Fully Paid           99999999.0  Short Term         741.0      2231892.0   
3  Fully Paid             347666.0   Long Term         721.0       806949.0   
4  Fully Paid             176220.0  Short Term           NaN            NaN   

  Years in current job H

/var/folders/mk/08bvt1ts40n3x2dxbq7s01680000gn/T/ipykernel_73917/2183066735.py:77: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_imputed[NUMBER_CREDIT_PRO].fillna(data_imputed[NUMBER_CREDIT_PRO].mode()[0], inplace=True)


^^^^^^^^^^^^^^^^^^^ Missing values after imputation:
Current Loan Amount                 0
Credit Score                        0
Annual Income                       0
Monthly Debt                        0
Years of Credit History             0
Months since last delinquent        0
Number of Open Accounts             0
Number of Credit Problems           0
Current Credit Balance              0
Maximum Open Credit                 0
Bankruptcies                        0
Tax Liens                           0
Loan ID                           514
Customer ID                       514
Loan Status                       514
Term                              514
Years in current job            10124
Home Ownership                    514
Purpose                           514
dtype: int64
^^^^^^^^^^^^^^^^^^^ Unique values in NUMBER_CREDIT_PRO column after cleaning:
[0. 1.]
^^^^^^^^^^^^^^^^^^^ Cleaned data has been saved to 'cleaned/dataset-for-bank-loan-prediction_cleaned.csv'
